In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

In [2]:
# Load the data
wine_data = pd.read_csv('winedata.csv').values # 300x12 2D np.array

# Standardize the data
wine_data = (wine_data - np.mean(wine_data, axis=0)) / np.std(wine_data, axis=0) # 300x12 np.array

# K-Means Algorithm

In [3]:
def k_means(data, c1, c2):
    '''
    Inputs: data, initial 2 cluster centers
    Returns:
    
    '''
    # Concatenate the two cluster centers
    cluster_centers = np.concatenate((c1, c2), axis=0) # 2x12
    
    # Initialize the previous cluster assignments to be different from the current ones
    prev_assignments = np.zeros(data.shape[0])
    
    # Initialize the current cluster assignments to be different from the previous ones
    assignments = np.ones(data.shape[0])
    
    num_iter = 0
    
    # Loop until the cluster assignments no longer change
    while not np.array_equal(prev_assignments, assignments):
        distances = np.linalg.norm(data - cluster_centers[:, np.newaxis], axis=2) # data - cluster_centers[:,np.newaxis] = 300x12 - 2x1x12 = 2x300x12
        assignments = np.argmin(distances, axis=0) # assignments of each data point to a cluster center (300 elements)
        
        for j in range(2):
            cluster_centers[j] = np.mean(data[assignments == j], axis=0) # update cluster centers based on the mean of the assigned data points
        
        # Update the previous cluster assignments
        prev_assignments = assignments
        
        # Increment the number of iterations
        num_iter += 1
    
    a = (np.sum(assignments[:150] == 1) + np.sum(assignments[150:] == 0)) / data.shape[0]
    b = (np.sum(assignments[:150] == 0) + np.sum(assignments[150:] == 1)) / data.shape[0]
    cluster_error = 1 - np.max(np.array([a,b]))
    
    return cluster_error, num_iter # return the final cluster centers, and the number of iterations

In [4]:
c1 = np.random.normal(0,1,size=(1,12))
c2 = np.random.normal(0,1,size=(1,12))

k_means(wine_data, c1, c2)

(0.25, 1)

In [5]:
c1 = np.mean(wine_data[:150,:], axis=0).reshape((1,12))
c2 = np.mean(wine_data[150:,:], axis=0).reshape((1,12))

k_means(wine_data, c1, c2)

(0.030000000000000027, 1)

# EM Algorithm for GMM

In [6]:
def EM_GMM(X, pi, mu, cov, max_iter):

    for i in range(max_iter):
        # E-step
        likelihood1 = multivariate_normal.pdf(X, mean=mu[0], cov=cov[0])
        likelihood2 = multivariate_normal.pdf(X, mean=mu[1], cov=cov[1])
        w1 = (pi[0] * likelihood1) / (pi[0] * likelihood1 + pi[1] * likelihood2) # posterior probability
        w2 = 1 - w1

        # M-step
        N1 = np.sum(w1)
        N2 = np.sum(w2)
        pi = np.array([N1, N2]) / X.shape[0]
        mu[0] = np.dot(w1, X) / N1
        mu[1] = np.dot(w2, X) / N2
        cov[0] = np.dot(w1 * (X - mu[0]).T, X - mu[0]) / (N1 - 1)
        cov[1] = np.dot(w1 * (X - mu[1]).T, X - mu[1]) / (N2 - 1)
        cov[0] = np.dot((X - mu[0]).T, (X - mu[0]) * w1.reshape(-1, 1)) / (N1 - 1)
        cov[1] = np.dot((X - mu[1]).T, (X - mu[1]) * w2.reshape(-1, 1)) / (N2 - 1)

    # compute the cluster error
    pred_labels = np.argmax(np.vstack([w1, w2]).T, axis=1)
    # true_labels = np.repeat([0, 1], X.shape[0]/2)
    
    a = (np.sum(pred_labels[:150] == 1) + np.sum(pred_labels[150:] == 0)) / X.shape[0]
    b = (np.sum(pred_labels[:150] == 0) + np.sum(pred_labels[150:] == 1)) / X.shape[0]
    cluster_error = 1 - np.max(np.array([a,b]))
    
    return cluster_error

In [7]:
pi = np.array([0.5, 0.5])

mean1 = np.random.normal(0,1,size=(1,12))
mean2 = np.random.normal(0,1,size=(1,12))
mu = np.vstack([mean1, mean2])

cov = np.tile((np.eye(12)), (2, 1, 1))

EM_GMM(wine_data, pi, mu, cov, 400)

0.4666666666666667

In [8]:
pi = np.array([0.5, 0.5])

mean1 = np.mean(wine_data[:150,:], axis=0).reshape((1,12))
mean2 = np.mean(wine_data[150:,:], axis=0).reshape((1,12))
mu = np.vstack([mean1, mean2])

cov = np.tile((np.eye(12)), (2, 1, 1))

EM_GMM(wine_data, pi, mu, cov, 400)

0.036666666666666625